In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn import preprocessing
from sklearn.svm import LinearSVC,SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.neural_network import MLPClassifier


In [2]:
data = pd.read_csv('Assign1_Training_Data.txt',delimiter="\t")
data.head()
d = np.asarray(data)
data.head()

,Sample_Number,AL080059,Contig63649_RC,Contig46218_RC,LOC51203,AA555029_RC,ALDH4,Contig38288_RC,FGF18,Contig28552_RC,...,AKAP2,Contig63102_RC,PRC1,Contig20217_RC,CENPA,SM.20,CCNE2,ESM1,FLJ11190,Label
0,138,-0.227,-0.107,-0.086,-0.057,0.073,0.021,-0.002,0.135,-0.071,...,0.121,-0.187,-0.200,-0.127,-0.277,0.036,-0.095,0.123,-0.211,0
1,184,0.044,-0.031,0.381,0.226,-0.038,-0.167,0.103,-0.330,0.420,...,0.029,0.033,0.333,0.303,0.058,0.406,0.266,-0.129,-0.219,0
2,127,0.151,-0.210,0.034,0.037,-0.065,-0.048,-0.026,-0.425,0.204,...,-0.200,-0.070,0.166,-0.002,0.076,-0.027,0.278,-0.160,-0.144,0
3,166,0.335,-0.031,0.177,0.165,-0.372,0.340,0.112,-0.506,0.213,...,-0.251,-0.040,0.058,0.074,0.309,-0.227,0.186,0.104,0.214,0
4,318,-0.098,-0.492,-0.307,-0.097,-0.024,0.197,-0.001,0.369,-0.318,...,-0.003,0.178,-0.453,-0.285,-0.265,-0.237,-0.335,-0.603,-0.116,0


In [3]:
data_testing = pd.read_csv('Assign1_Testing_Data.txt',delimiter="\t")
d_test = np.asarray(data_testing)
Y_test =   d_test[:,-1]
X_test =   d_test[:,1:71]

x0_test = []
y0_test = []
x1_test = []
y1_test = []
for i in range(len(Y_test)):
    if Y_test[i]==0:
        x0_test.append(X_test[i])
        y0_test.append(Y_test[i])
    else:
        x1_test.append(X_test[i])
        y1_test.append(Y_test[i])
        
print('Number of  0 class in Test set is =',len(x0_test))
print('Number of  1 class in Test set is',len(x1_test))
print('Number of  Total classes in Test set is=',len(X_test))

Number of  0 class in Test set is = 39
Number of  1 class in Test set is 176
Number of  Total classes in Test set is= 215


In [4]:
Y =   d[:,-1]
X =   d[:,1:71]
print('Some Features --> ',X[0:5,0:3])
p = np.vstack((X[:,0], X[:,1],X[:,2])).T
print(X[:,2].shape)
#p = np.vstack((p, )).T
#p.shape
#X.shape
#p
x0 = []
y0 = []
x1 = []
y1 = []
for i in range(len(Y)):
    if Y[i]==0:
        x0.append(X[i])
        y0.append(Y[i])
    else:
        x1.append(X[i])
        y1.append(Y[i])
        
print('Number of  0 class in Train set is',len(x0))
print('Number of  1 class in Train set is',len(x1))
print('Number of  Total classes in Train set is',len(X))

Some Features -->  [[-0.227 -0.107 -0.086]
 [ 0.044 -0.031  0.381]
 [ 0.151 -0.21   0.034]
 [ 0.335 -0.031  0.177]
 [-0.098 -0.492 -0.307]]
(80,)
Number of  0 class in Train set is 40
Number of  1 class in Train set is 40
Number of  Total classes in Train set is 80


In [5]:
x0 = np.array(x0)
x1 = np.array(x1)
x0_test = np.array(x0_test)
x1_test = np.array(x1_test)

In [6]:
def fs_es(X,Y,clf,x0,x0_test,y0,y0_test,x1,x1_test,y1,y1_test,X_test,Y_test):
    c = X.shape[1] 
    temp_e = np.inf
    e = 0
    for i in range(c):
        for j in range(i+1,c):
            x = np.vstack((X[:,i], X[:,j])).T
            y = Y
            Xx_train, Xx_test, yy_train, yy_test = train_test_split(x,y, test_size=0.0, random_state=0)
            e = clf.fit(Xx_train, yy_train).predict(Xx_train)
            e = np.average(abs(yy_train - e))
            #print(e)
            if e < temp_e:
                index1 = i
                index2 = j
                temp_e = e
    x0_ = np.vstack((x0[:,index1],x0[:,index2])).T
    x1_ = np.vstack((x1[:,index1],x1[:,index2])).T   
    x0_test_ = np.vstack((x0_test[:,index1],x0_test[:,index2])).T
    x1_test_ = np.vstack((x1_test[:,index1],x1_test[:,index2])).T

    
    x_test_ = np.vstack((X_test[:,index1],X_test[:,index2])).T
    
    e0_train  = clf.predict(x0_)
    e0_train  = np.average(abs(y0 - e0_train))
    e1_train  = clf.predict(x1_)
    e1_train  = np.average(abs(y1 - e1_train))
        
    e0_test  = clf.predict(x0_test_)
    e0_test  = np.average(abs(y0_test - e0_test))
    e1_test  = clf.predict(x1_test_)
    e1_test  = np.average(abs(y1_test - e1_test))
    
    e1_total  = clf.predict(x_test_)
    e1_total  = np.average(abs(Y_test - e1_total))
    return temp_e,index1,index2,e0_train,e1_train,e0_test,e1_test,e1_total      

In [7]:
def fs_sfs3(ind1,ind2,X,Y,clf,x0,x0_test,y0,y0_test,x1,x1_test,y1,y1_test,X_test,Y_test):
    c = X.shape[1] 
    
    temp_e = np.inf
    e = 0
    for i in range(c):
        if (i != ind1 and i != ind2):
            x = np.vstack((X[:,ind1], X[:,ind2], X[:,i])).T
            y = Y
            Xx_train, Xx_test, yy_train, yy_test = train_test_split(x,y, test_size=0.0, random_state=0)
            e = clf.fit(Xx_train, yy_train).predict(Xx_train)
            e = np.average(abs(yy_train - e))  
            #print(e)
            if e < temp_e:
                index = i
                temp_e = e    
    x0_ = np.vstack((x0[:,ind1],x0[:,ind2],x0[:,index])).T
    x1_ = np.vstack((x1[:,ind1],x1[:,ind2],x1[:,index])).T    
    x0_test_ = np.vstack((x0_test[:,ind1],x0_test[:,ind2],x0_test[:,index])).T
    x1_test_ = np.vstack((x1_test[:,ind1],x1_test[:,ind2],x1_test[:,index])).T
    x_test_ = np.vstack((X_test[:,ind1],X_test[:,ind2],X_test[:,index])).T
    
    
    e0_train  = clf.predict(x0_)
    e0_train  = np.average(abs(y0 - e0_train))
    e1_train  = clf.predict(x1_)
    e1_train  = np.average(abs(y1 - e1_train))
        
    e0_test  = clf.predict(x0_test_)
    e0_test  = np.average(abs(y0_test - e0_test))
    e1_test  = clf.predict(x1_test_)
    e1_test  = np.average(abs(y1_test - e1_test))
    
    e1_total  = clf.predict(x_test_)
    e1_total  = np.average(abs(Y_test - e1_total))

    return temp_e,index,e0_train,e1_train,e0_test,e1_test,e1_total         

In [8]:
def fs_sfs4(ind1,ind2,ind3,X,Y,clf,x0,x0_test,y0,y0_test,x1,x1_test,y1,y1_test,X_test,Y_test):
    c = X.shape[1] 
    
    temp_e = np.inf
    e = 0
    for i in range(c):
        if (i != ind1 and i != ind2 and i != ind3):
            x = np.vstack((X[:,ind1], X[:,ind2], X[:,ind3],X[:,i])).T
            y = Y
            Xx_train, Xx_test, yy_train, yy_test = train_test_split(x,y, test_size=0.0, random_state=0)
            e = clf.fit(Xx_train, yy_train).predict(Xx_train)
            e = np.average(abs(yy_train - e))    
            #print(e)
            if e < temp_e:
                index = i
                temp_e = e 
                
    x0_ = np.vstack((x0[:,ind1],x0[:,ind2],x0[:,ind3],x0[:,index])).T
    x1_ = np.vstack((x1[:,ind1],x1[:,ind2],x1[:,ind3],x1[:,index])).T
    
    x0_test_ = np.vstack((x0_test[:,ind1],x0_test[:,ind2],x0_test[:,ind3],x0_test[:,index])).T
    x1_test_ = np.vstack((x1_test[:,ind1],x1_test[:,ind2],x1_test[:,ind3],x1_test[:,index])).T
    
    x_test_ = np.vstack((X_test[:,ind1],X_test[:,ind2],X_test[:,ind3],X_test[:,index])).T
    
    e0_train  = clf.predict(x0_)
    e0_train  = np.average(abs(y0 - e0_train))
    e1_train  = clf.predict(x1_)
    e1_train  = np.average(abs(y1 - e1_train))
        
    e0_test  = clf.predict(x0_test_)
    e0_test  = np.average(abs(y0_test - e0_test))
    e1_test  = clf.predict(x1_test_)
    e1_test  = np.average(abs(y1_test - e1_test))
    
    e1_total  = clf.predict(x_test_)
    e1_total  = np.average(abs(Y_test - e1_total))
    
    return temp_e,index,e0_train,e1_train,e0_test,e1_test,e1_total   

In [9]:
def fs_sfs5(ind1,ind2,ind3,ind4,X,Y,clf,x0,x0_test,y0,y0_test,x1,x1_test,y1,y1_test,X_test,Y_test):
    c = X.shape[1] 
    
    temp_e = np.inf
    e = 0
    for i in range(c):
        if (i != ind1 and i != ind2 and i != ind3 and i != ind4):
            x = np.vstack((X[:,ind1], X[:,ind2], X[:,ind3],X[:,ind4],X[:,i])).T
            y = Y
            Xx_train, Xx_test, yy_train, yy_test = train_test_split(x,y, test_size=0.0, random_state=0)
            e = clf.fit(Xx_train, yy_train).predict(Xx_train)
            e = np.average(abs(yy_train - e)) 
            #print(e)
            if e < temp_e:
                index = i
                temp_e = e    
    x0_ = np.vstack((x0[:,ind1],x0[:,ind2],x0[:,ind3],x0[:,ind4],x0[:,index])).T
    x1_ = np.vstack((x1[:,ind1],x1[:,ind2],x1[:,ind3],x1[:,ind4],x1[:,index])).T
    
    x0_test_ = np.vstack((x0_test[:,ind1],x0_test[:,ind2],x0_test[:,ind3],x0_test[:,ind4],x0_test[:,index])).T
    x1_test_ = np.vstack((x1_test[:,ind1],x1_test[:,ind2],x1_test[:,ind3],x1_test[:,ind4],x1_test[:,index])).T
    
    x_test_ = np.vstack((X_test[:,ind1],X_test[:,ind2],X_test[:,ind3],X_test[:,ind4],X_test[:,index])).T
    
    e0_train  = clf.predict(x0_)
    e0_train  = np.average(abs(y0 - e0_train))
    e1_train  = clf.predict(x1_)
    e1_train  = np.average(abs(y1 - e1_train))
        
    e0_test  = clf.predict(x0_test_)
    e0_test  = np.average(abs(y0_test - e0_test))
    e1_test  = clf.predict(x1_test_)
    e1_test  = np.average(abs(y1_test - e1_test))
    
    e1_total  = clf.predict(x_test_)
    e1_total  = np.average(abs(Y_test - e1_total))
    
    return temp_e,index,e0_train,e1_train,e0_test,e1_test,e1_total

# 1) Linear SVM

#### 1.1 Linear SVM NO feature selection 

In [10]:
clf_svmL = SVC(kernel='linear', C=1)
Xn_train, Xn_test, yn_train, yn_test = train_test_split(X,Y, test_size=0.0, random_state=0)
e = 1 - clf_svmL.fit(Xn_train, yn_train).score(Xn_train, yn_train)
e0_train_svm2_n = 1-clf_svmL.score(x0, y0)
e1_train_svm2_n = 1-clf_svmL.score(x1, y1)
e0_test_svm2_n  = 1-clf_svmL.score(x0_test, y0_test)  
e1_test_svm2_n  = 1-clf_svmL.score(x1_test, y1_test)
e1_total_svm2_n = 1-clf_svmL.score(X_test, Y_test)
print('Resubstitution error = ',e)
print("Train Error on class 0",e0_train_svm2_n)
print("Train Error on class 1",e1_train_svm2_n)
print("Test Error on class 0",e0_test_svm2_n)
print("Test Error on class 1",e1_test_svm2_n)
print("Test Error Total",e1_total_svm2_n)

Resubstitution error =  0.050000000000000044
Train Error on class 0 0.07499999999999996
Train Error on class 1 0.025000000000000022
Test Error on class 0 0.3846153846153846
Test Error on class 1 0.3295454545454546
Test Error Total 0.3395348837209302


#### 1.2 Linear SVM 2-feature selection extraction 

In [11]:

e,feature1_svmL,feature2_svmL,e0_train_svm2_2,e1_train_svm2_2,e0_test_svm2_2,e1_test_svm2_2,e1_total_svm2_2  = fs_es(X,Y,clf_svmL,x0,x0_test,y0,
                                                                                 y0_test,x1,x1_test,y1,y1_test,X_test,Y_test)
print('Resubstitution error = ',e,'\nFeature 1 = ', feature1_svmL,' Feature 2 = ',feature2_svmL)
print("Train Error on class 0",e0_train_svm2_2)
print("Train Error on class 1",e1_train_svm2_2)
print("Test Error on class 0",e0_test_svm2_2)
print("Test Error on class 1",e1_test_svm2_2)
print("Test Error Total",e1_total_svm2_2)

Resubstitution error =  0.2125 
Feature 1 =  3  Feature 2 =  19
Train Error on class 0 0.325
Train Error on class 1 0.25
Test Error on class 0 0.41025641025641024
Test Error on class 1 0.3693181818181818
Test Error Total 0.3767441860465116


#### 1.3 Linear SVM 3-feature selection sequential forward search 

In [12]:
e,feature3_svmL,e0_train_svm2_3,e1_train_svm2_3,e0_test_svm2_3,e1_test_svm2_3,e1_total_svm2_3 = fs_sfs3(feature1_svmL,feature2_svmL,X,Y,clf_svmL,
                                                                                              x0,x0_test,y0,y0_test,x1,x1_test,y1,y1_test,X_test,Y_test)
print('Resubstitution error = ',e,'\nFeature 1 = ', feature1_svmL,' Feature 2 = ',feature2_svmL,' Feature 3 = ',feature3_svmL)
print("Train Error on class 0",e0_train_svm2_3)
print("Train Error on class 1",e1_train_svm2_3)
print("Test Error on class 0",e0_test_svm2_3)
print("Test Error on class 1",e1_test_svm2_3)
print("Test Error Total",e1_total_svm2_3)

Resubstitution error =  0.2 
Feature 1 =  3  Feature 2 =  19  Feature 3 =  27
Train Error on class 0 0.05
Train Error on class 1 0.45
Test Error on class 0 0.15384615384615385
Test Error on class 1 0.5738636363636364
Test Error Total 0.49767441860465117


#### 1.4 Linear SVM 4-feature selection sequential forward search 

In [13]:
e,feature4_svmL,e0_train_svm2_4,e1_train_svm2_4,e0_test_svm2_4,e1_test_svm2_4,e1_total_svm2_4 = fs_sfs4(feature1_svmL,feature2_svmL,feature3_svmL,X,Y,clf_svmL,
                                                                                              x0,x0_test,y0,y0_test,x1,x1_test,y1,y1_test,X_test,Y_test)
print('Resubstitution error = ',e,'\nFeature 1 = ', feature1_svmL,' Feature 2 = ',feature2_svmL,' Feature 3 = ',feature3_svmL,' Feature 4 = ',feature4_svmL)
print("Train Error on class 0",e0_train_svm2_4)
print("Train Error on class 1",e1_train_svm2_4)
print("Test Error on class 0",e0_test_svm2_4)
print("Test Error on class 1",e1_test_svm2_4)
print("Test Error Total",e1_total_svm2_4)

Resubstitution error =  0.1875 
Feature 1 =  3  Feature 2 =  19  Feature 3 =  27  Feature 4 =  5
Train Error on class 0 0.05
Train Error on class 1 0.425
Test Error on class 0 0.1282051282051282
Test Error on class 1 0.5454545454545454
Test Error Total 0.4697674418604651


#### 1.5 Linear SVM 5-feature selection sequential forward search 

In [14]:
e,feature5_svmL,e0_train_svm2_5,e1_train_svm2_5,e0_test_svm2_5,e1_test_svm2_5,e1_total_svm2_5 = fs_sfs5(feature1_svmL,feature2_svmL,feature3_svmL,feature4_svmL,X,Y,clf_svmL,
                                                                                              x0,x0_test,y0,y0_test,x1,x1_test,y1,y1_test,X_test,Y_test)
print('Resubstitution error = ',e,'\nFeature 1 = ', feature1_svmL,' Feature 2 = ',feature2_svmL,' Feature 3 = ',feature3_svmL,' Feature 4 = ',feature4_svmL,' Feature 5 = ',feature5_svmL)
print("Train Error on class 0",e0_train_svm2_5)
print("Train Error on class 1",e1_train_svm2_5)
print("Test Error on class 0",e0_test_svm2_5)
print("Test Error on class 1",e1_test_svm2_5)
print("Test Error Total",e1_total_svm2_5)

Resubstitution error =  0.1875 
Feature 1 =  3  Feature 2 =  19  Feature 3 =  27  Feature 4 =  5  Feature 5 =  44
Train Error on class 0 0.1
Train Error on class 1 0.375
Test Error on class 0 0.15384615384615385
Test Error on class 1 0.5284090909090909
Test Error Total 0.4604651162790698


# 2) Non Linear SVM 

#### 2.1 Non Linear SVM no feature selection

In [15]:
clf_svmNL = SVC(gamma ='auto' ,kernel='rbf', C=10)
Xn_train, Xn_test, yn_train, yn_test = train_test_split(X,Y, test_size=0.0, random_state=0)
e = 1-clf_svmNL.fit(Xn_train, yn_train).score(Xn_train, yn_train)
e0_trainsvmNL_n = 1-clf_svmNL.score(x0, y0)
e1_trainsvmNL_n = 1-clf_svmNL.score(x1, y1)
e0_testsvmNL_n  = 1-clf_svmNL.score(x0_test, y0_test)  
e1_testsvmNL_n  = 1-clf_svmNL.score(x1_test, y1_test)
e1_totalsvmNL_n = 1-clf_svmNL.score(X_test, Y_test)
print('Resubstitution error = ',e)
print("Train Error on class 0",e0_trainsvmNL_n)
print("Train Error on class 1",e1_trainsvmNL_n)
print("Test Error on class 0",e0_testsvmNL_n)
print("Test Error on class 1",e1_testsvmNL_n)
print("Test Error Total",e1_totalsvmNL_n)

Resubstitution error =  0.0625
Train Error on class 0 0.09999999999999998
Train Error on class 1 0.025000000000000022
Test Error on class 0 0.3846153846153846
Test Error on class 1 0.3352272727272727
Test Error Total 0.3441860465116279


#### 2.2 Non-Linear SVM 2-feature selection extraction 

In [16]:

e,feature1_svmNL,feature2_svmNL,e0_train_svmNL_2,e1_train_svmNL_2,e0_test_svmNL_2,e1_test_svmNL_2,e1_total_svmNL_2  = fs_es(X,Y,clf_svmNL,x0,x0_test,y0,
                                                                                 y0_test,x1,x1_test,y1,y1_test,X_test,Y_test)
print('Resubstitution error = ',e,'\nFeature 1 = ', feature1_svmNL,' Feature 2 = ',feature2_svmNL)
print("Train Error on class 0",e0_train_svmNL_2)
print("Train Error on class 1",e1_train_svmNL_2)
print("Test Error on class 0",e0_test_svmNL_2)
print("Test Error on class 1",e1_test_svmNL_2)
print("Test Error Total",e1_total_svmNL_2)

Resubstitution error =  0.2125 
Feature 1 =  12  Feature 2 =  48
Train Error on class 0 0.325
Train Error on class 1 0.425
Test Error on class 0 0.1794871794871795
Test Error on class 1 0.4602272727272727
Test Error Total 0.40930232558139534


#### 2.3 Non Linear SVM 3-feature selection sequential forward search 

In [17]:
e,feature3_svmNL,e0_train_svmNL_3,e1_train_svmNL_3,e0_test_svmNL_3,e1_test_svmNL_3,e1_total_svmNL_3 = fs_sfs3(feature1_svmNL,feature2_svmNL,X,Y,clf_svmNL,
                                                                                              x0,x0_test,y0,y0_test,x1,x1_test,y1,y1_test,X_test,Y_test)
print('Resubstitution error = ',e,'\nFeature 1 = ', feature1_svmNL,' Feature 2 = ',feature2_svmNL,' Feature 3 = ',feature3_svmNL)
print("Train Error on class 0",e0_train_svmNL_3)
print("Train Error on class 1",e1_train_svmNL_3)
print("Test Error on class 0",e0_test_svmNL_3)
print("Test Error on class 1",e1_test_svmNL_3)
print("Test Error Total",e1_total_svmNL_3)

Resubstitution error =  0.175 
Feature 1 =  12  Feature 2 =  48  Feature 3 =  32
Train Error on class 0 0.625
Train Error on class 1 0.275
Test Error on class 0 0.6410256410256411
Test Error on class 1 0.36363636363636365
Test Error Total 0.413953488372093


#### 2.4 Non Linear SVM 4-feature selection sequential forward search 

In [18]:
e ,feature4_svmNL,e0_train_svmNL_4,e1_train_svmNL_4,e0_test_svmNL_4,e1_test_svmNL_4,e1_total_svmNL_4 = fs_sfs4(feature1_svmNL,feature2_svmNL,feature3_svmNL,X,Y,clf_svmNL,
                                                                                              x0,x0_test,y0,y0_test,x1,x1_test,y1,y1_test,X_test,Y_test)
print('Resubstitution error = ',e,'\nFeature 1 = ', feature1_svmNL,' Feature 2 = ',feature2_svmNL,' Feature 3 = ',feature3_svmNL,' Feature 4 = ',feature4_svmNL)
print("Train Error on class 0",e0_train_svmNL_4)
print("Train Error on class 1",e1_train_svmNL_4)
print("Test Error on class 0",e0_test_svmNL_4)
print("Test Error on class 1",e1_test_svmNL_4)
print("Test Error Total",e1_total_svmNL_4)

Resubstitution error =  0.15 
Feature 1 =  12  Feature 2 =  48  Feature 3 =  32  Feature 4 =  1
Train Error on class 0 0.1
Train Error on class 1 0.25
Test Error on class 0 0.23076923076923078
Test Error on class 1 0.48863636363636365
Test Error Total 0.4418604651162791


#### 2.5 Non Linear SVM 5-feature selection sequential forward search  

In [19]:
e ,feature5_svmNL,e0_train_svmNL_5,e1_train_svmNL_5,e0_test_svmNL_5,e1_test_svmNL_5,e1_total_svmNL_5 = fs_sfs5(feature1_svmNL,feature2_svmNL,feature3_svmNL,feature4_svmNL,X,Y,clf_svmNL,
                                                                                              x0,x0_test,y0,y0_test,x1,x1_test,y1,y1_test,X_test,Y_test)
print('Resubstitution error = ',e,'\nFeature 1 = ', feature1_svmNL,' Feature 2 = ',feature2_svmNL,' Feature 3 = ',feature3_svmNL,' Feature 4 = ',feature4_svmNL,' Feature 5 = ',feature5_svmNL)
print("Train Error on class 0",e0_train_svmNL_5)
print("Train Error on class 1",e1_train_svmNL_5)
print("Test Error on class 0",e0_test_svmNL_5)
print("Test Error on class 1",e1_test_svmNL_5)
print("Test Error Total",e1_total_svmNL_5)

Resubstitution error =  0.1375 
Feature 1 =  12  Feature 2 =  48  Feature 3 =  32  Feature 4 =  1  Feature 5 =  24
Train Error on class 0 0.125
Train Error on class 1 0.25
Test Error on class 0 0.2564102564102564
Test Error on class 1 0.4431818181818182
Test Error Total 0.40930232558139534


# 3 Neural Network

#### 3.1 Neural Network No feature selection  

In [20]:
clf_NN = MLPClassifier((5,5),solver = 'lbfgs',activation='relu',
                     learning_rate= 'constant',learning_rate_init=0.001,
                    random_state=0,alpha=0.0001,)
Xn_train, Xn_test, yn_train, yn_test = train_test_split(X,Y, test_size=0.0, random_state=0)
e = 1 - clf_NN.fit(Xn_train, yn_train).score(Xn_train, yn_train)
e0_train_NN_n = 1-clf_NN.score(x0, y0)
e1_train_NN_n = 1-clf_NN.score(x1, y1)
e0_test_NN_n  = 1-clf_NN.score(x0_test, y0_test)  
e1_test_NN_n  = 1-clf_NN.score(x1_test, y1_test)
e1_total_NN_n = 1-clf_NN.score(X_test, Y_test)
print('Resubstitution error = ',e)
print("Train Error on class 0",e0_train_NN_n)
print("Train Error on class 1",e1_train_NN_n)
print("Test Error on class 0",e0_test_NN_n)
print("Test Error on class 1",e1_test_NN_n)
print("Test Error Total",e1_total_NN_n)

Resubstitution error =  0.0
Train Error on class 0 0.0
Train Error on class 1 0.0
Test Error on class 0 0.33333333333333337
Test Error on class 1 0.34090909090909094
Test Error Total 0.3395348837209302


#### 3.2 Neural Network 2-feature selection extraction search

In [21]:

e,feature1_nn,feature2_nn,e0_train_nn_2,e1_train_nn_2,e0_test_nn_2,e1_test_nn_2,e1_total_nn_2  = fs_es(X,Y,clf_NN,x0,x0_test,y0,
                                                                                 y0_test,x1,x1_test,y1,y1_test,X_test,Y_test)
print('Resubstitution error = ',e,'\nFeature 1 = ', feature1_nn,' Feature 2 = ',feature2_nn)
print("Train Error on class 0",e0_train_nn_2)
print("Train Error on class 1",e1_train_nn_2)
print("Test Error on class 0",e0_test_nn_2)
print("Test Error on class 1",e1_test_nn_2)
print("Test Error Total",e1_total_nn_2)

Resubstitution error =  0.15 
Feature 1 =  59  Feature 2 =  67
Train Error on class 0 0.425
Train Error on class 1 0.225
Test Error on class 0 0.41025641025641024
Test Error on class 1 0.3977272727272727
Test Error Total 0.4


#### Neural Network 3-feature selection sequential forward search 

In [22]:
e,feature3_nn,e0_train_nn_3,e1_train_nn_3,e0_test_nn_3,e1_test_nn_3,e1_total_nn_3 = fs_sfs3(feature1_nn,feature2_nn,X,Y,clf_NN,
                                                                                              x0,x0_test,y0,y0_test,x1,x1_test,y1,y1_test,X_test,Y_test)
print('Resubstitution error = ',e,'\nFeature 1 = ', feature1_nn,' Feature 2 = ',feature2_nn,' Feature 3 = ',feature3_nn)
print("Train Error on class 0",e0_train_nn_3)
print("Train Error on class 1",e1_train_nn_3)
print("Test Error on class 0",e0_test_nn_3)
print("Test Error on class 1",e1_test_nn_3)
print("Test Error Total",e1_total_nn_3)

Resubstitution error =  0.125 
Feature 1 =  59  Feature 2 =  67  Feature 3 =  51
Train Error on class 0 0.375
Train Error on class 1 0.225
Test Error on class 0 0.38461538461538464
Test Error on class 1 0.38636363636363635
Test Error Total 0.386046511627907


#### Neural Network 4-feature selection sequential forward search  

In [23]:
e,feature4_nn,e0_train_nn_4,e1_train_nn_4,e0_test_nn_4,e1_test_nn_4,e1_total_nn_4 = fs_sfs4(feature1_nn,feature2_nn,feature3_nn,X,Y,clf_NN,
                                                                                              x0,x0_test,y0,y0_test,x1,x1_test,y1,y1_test,X_test,Y_test)
print('Resubstitution error = ',e,'\nFeature 1 = ', feature1_nn,' Feature 2 = ',feature2_nn,' Feature 3 = ',feature3_nn,' Feature 4 = ',feature4_nn)
print("Train Error on class 0",e0_train_nn_4)
print("Train Error on class 1",e1_train_nn_4)
print("Test Error on class 0",e0_test_nn_4)
print("Test Error on class 1",e1_test_nn_4)
print("Test Error Total",e1_total_nn_4)

Resubstitution error =  0.0375 
Feature 1 =  59  Feature 2 =  67  Feature 3 =  51  Feature 4 =  19
Train Error on class 0 0.25
Train Error on class 1 0.45
Test Error on class 0 0.3076923076923077
Test Error on class 1 0.4659090909090909
Test Error Total 0.4372093023255814


#### Neural network 5-feature selection sequential forward search  

In [24]:
e,feature5_nn,e0_train_nn_5,e1_train_nn_5,e0_test_nn_5,e1_test_nn_5,e1_total_nn_5 = fs_sfs5(feature1_nn,feature2_nn,feature3_nn,feature4_nn,X,Y,clf_NN,
                                                                                              x0,x0_test,y0,y0_test,x1,x1_test,y1,y1_test,X_test,Y_test)
print('Resubstitution error = ',e,'\nFeature 1 = ', feature1_nn,' Feature 2 = ',feature2_nn,' Feature 3 = ',feature3_nn,' Feature 4 = ',feature4_nn,' Feature 5 = ',feature5_nn)
print("Train Error on class 0",e0_train_nn_5)
print("Train Error on class 1",e1_train_nn_5)
print("Test Error on class 0",e0_test_nn_5)
print("Test Error on class 1",e1_test_nn_5)
print("Test Error Total",e1_total_nn_5)

Resubstitution error =  0.0 
Feature 1 =  59  Feature 2 =  67  Feature 3 =  51  Feature 4 =  19  Feature 5 =  30
Train Error on class 0 0.15
Train Error on class 1 0.2
Test Error on class 0 0.38461538461538464
Test Error on class 1 0.4147727272727273
Test Error Total 0.40930232558139534
